In [1]:
import pandas as pd
import sqlite3

In [2]:
# data for state codes
states = pd.read_excel('./CRA Assessment Tracts/state_codes.xlsx', dtype={'State Code':str}).drop(columns='State Name')

# dataframe to concat annual sheets
tracts = pd.DataFrame()

for year in [2018, 2019, 2020, 2021]:
    # read data
    df = pd.read_excel(f'./CRA Assessment Tracts/HSBC_{year}.XLSX', 
                    usecols='A', skiprows=4)
    df.dropna(inplace=True)
    df.rename(columns={df.columns[0]:'data'}, inplace=True)

    # create columns
    df.loc[df.data.str.contains('ASSESSMENT AREA'), 'assessment_area'] = df.loc[df.data.str.contains('ASSESSMENT AREA')].data
    df.loc[(df.data.str.contains('COUNTY'))|(df.data.str.contains('DISTRICT OF')), 'county'] = df.loc[(df.data.str.contains('COUNTY'))|(df.data.str.contains('DISTRICT OF'))].data
    df.loc[df.data.str.contains('MSA:'), 'msa'] = df.loc[df.data.str.contains('MSA:')].data
    df['year'] = year

    # drop unneeded subheads
    df = df.loc[~df.data.str.contains('Income')]

    # use values from created columns
    df.fillna(method='ffill', inplace=True)
    # drop rows used to create values
    df = df.loc[~((df.data.str.contains('COUNTY'))|(df.data.str.contains('DISTRICT OF'))|(df.data.str.contains('ASSESSMENT'))
            |(df.data.str.contains('Median Family Income'))|(df.data.str.contains('MSA'))|(df.data.str.contains('Income')))].reset_index(drop=True)

    # clean columns
    df.loc[:, 'data'] = df.data.map(lambda x: [y.rstrip('*') for y in x.split()])
    df.loc[:, 'msa'] = df.msa.str.lstrip('MSA: ')
    df['state'] = df.county.map(lambda x: x.split()[-1])
    df['county_code'] = df.county.map(lambda x: x.split(',')[0].split()[-1].replace('(','').replace(')',''))
    df.loc[:, 'county'] = df.county.map(lambda x: x.split('(')[0].strip())

    # add state codes
    df = df.merge(states, on='state')

    tracts = pd.concat([tracts, df], ignore_index=True)

# create row for each tract in data column
final = pd.DataFrame()
for idx in tracts.index:
    row = tracts.iloc[idx]
    for tract in row.data:
        final = pd.concat([final, 
            pd.DataFrame(dict(zip(
                ['tract', 'assessment_area', 'county', 'state', 'msa', 'county_code', 'state_code', 'census_tract', 'year'], 
                [[tract.replace('.','')], [row.assessment_area], [row.county], [row.state], [row.msa], [row.county_code], [row['State Code']], 
                [row['State Code'] + row.county_code + tract.replace('.','')], [row.year]]
                ))
            )],
            ignore_index=True
        )

final.info()

c:\Users\WilliamRobinson\miniconda3\Lib\site-packages\openpyxl\styles\stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
c:\Users\WilliamRobinson\miniconda3\Lib\site-packages\openpyxl\styles\stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
c:\Users\WilliamRobinson\miniconda3\Lib\site-packages\openpyxl\styles\stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
c:\Users\WilliamRobinson\miniconda3\Lib\site-packages\openpyxl\styles\stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 58326 entries, 0 to 58325
Data columns (total 9 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   tract            58326 non-null  object
 1   assessment_area  58326 non-null  object
 2   county           58326 non-null  object
 3   state            58326 non-null  object
 4   msa              58326 non-null  object
 5   county_code      58326 non-null  object
 6   state_code       58326 non-null  object
 7   census_tract     58326 non-null  object
 8   year             58326 non-null  int64 
dtypes: int64(1), object(8)
memory usage: 4.0+ MB


In [3]:
print('census tracts:', len(final.loc[final.year==2018, 'census_tract'].unique()))
print('msas:', len(final.loc[final.year==2018, 'msa'].unique()))
print('assessment_areas:', len(final.loc[final.year==2018, 'assessment_area'].unique()))

census tracts: 13253
msas: 142
assessment_areas: 20


In [4]:
# save as sql table
with sqlite3.connect('../HMDA_full/HMDA.sqlite') as conn:
    cur = conn.cursor()
    # set schema
    cur.executescript('''
    DROP TABLE IF EXISTS hsbc_cra;

    CREATE TABLE "hsbc_cra" (
        "tract"	            TEXT,
        "assessment_area"	TEXT,
        "county"	        TEXT,
        "state"	            TEXT,
        "msa"	            TEXT,
        "county_code"	    TEXT,
        "state_code"	    TEXT,
        "census_tract"	    TEXT,
        "year"	            INTEGER
    );
    ''')
    # save data
    final.to_sql('hsbc_cra', conn, if_exists='replace', index=False)